## Kafka Producer

In [24]:
import time
import json
import kafka

In [25]:
topic = 'hello'
bootstrap_server = "localhost:9092"
producer = kafka.KafkaProducer(
    bootstrap_servers=[bootstrap_server],
    key_serializer=lambda key: key.encode('utf-8'),
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

In [4]:
# for i in range(1000):
#     data = {
#         "num": i
#     }
#     producer.send(topic, key="num", value=data)
#     time.sleep(3)

KeyboardInterrupt: 

## Get Stock Live Data

In [5]:
# !pip install yfinance

     ---------------------------------------- 59.7/59.7 kB 3.1 MB/s eta 0:00:00
     -------------------------------------- 112.2/112.2 kB 6.4 MB/s eta 0:00:00


In [28]:
import yfinance as yf
import datetime

In [8]:
df = yf.download(tickers='UBER', period='5d', interval='5m')

[*********************100%***********************]  1 of 1 completed


In [18]:
stock_dict = df["High"].to_dict().items()

In [33]:
for d in stock_dict:
    data = (d[0].strftime('%Y-%m-%d %H:%M:%S'), d[1])
    producer.send(topic, key="UBER", value=data)
    time.sleep(3)

KeyboardInterrupt: 